In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from IPython.display import clear_output, display

In [ ]:
%matplotlib inline

In [ ]:
df_data = pd.read_csv("full-stats.csv")

In [ ]:
df_data['home_diff'] = df_data['home_actual'] - df_data['home_expected']

# Analysis of the Home Team Wins and Losses/Draws counts vs Win Odds data for specific (user input) Home Difference range

In [ ]:
max_home_diff = df_data['home_diff'].max()
min_home_diff = df_data['home_diff'].min()
max_home_odds = df_data['home_odds'].max()
min_home_odds = df_data['home_odds'].min()

In [ ]:
# Define the win_odds ranges
home_odds_ranges = [(1, 1.8), (1.8, 2.05), (2.05, 2.5), (2.5, 3), (3, 4), (4, 5)]

# Create the sliders
home_diff_slider = widgets.FloatRangeSlider(value=[min_home_diff, max_home_diff],
                                          min=min_home_diff, max=max_home_diff, step=0.1, description='Home Difference %')
#home_odds_slider = widgets.FloatRangeSlider(value=[min_home_odds, max_home_odds,],
#                                            min=min_home_odds, max=max_home_odds, step=0.1, description='Home Odds')


# Define the update function
def update_chart(home_diff_range):

    # Initialize lists to store counts of wins and losses/draws for each range
    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the win_odds ranges
    for home_odds_range in home_odds_ranges:
        # Filter the data based on the win_odds range and home_diff threshold
        filtered_data = df_data[(df_data['home_diff'] >= home_diff_range[0]) &
                                               (df_data['home_diff'] <= home_diff_range[1]) &
                                               (df_data['home_odds'] >= home_odds_range[0]) &
                                               (df_data['home_odds'] <= home_odds_range[1])]

        # Calculate the counts of wins and losses/draws
        wins_count = filtered_data[filtered_data['home_team_win'] == 1]['home_team_win'].count()
        losses_draws_count = filtered_data[filtered_data['home_team_win'] == 0]['home_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(home_odds_ranges)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(home_odds_ranges)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')
    plt.xlabel('Win Odds Range')
    plt.xticks(range(len(home_odds_ranges)), [f'{home_odds_range[0]}-{home_odds_range[1]}' for home_odds_range in home_odds_ranges])
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nTotal Matches: {match_count}')
    plt.legend()
    plt.show()

# Create the interactive widget
interactive_widget = widgets.interactive(update_chart, home_diff_range=home_diff_slider)

# Display the widget
display(interactive_widget)

# Analysis of the Home Team Wins and Losses/Draws counts vs Home Actual Percentage data (> 60%, 70%, 75% and 80%) when Home Odds in the range 1.80 and 2.05 and Home Avg Rating > Away Avg Rating

In [ ]:
# Get the unique country and league names
unique_countries = df_data['country'].unique()

# Create the country and league selection widgets
countries_select = widgets.SelectMultiple(options=np.append('All', unique_countries), description='Countries:', rows=5)
leagues_select = widgets.SelectMultiple(options=[], description='Leagues:', rows=5)
apply_button = widgets.Button(description='Apply')
output = widgets.Output()

# Define the update leagues function
def update_leagues(*args):
    selected_countries = countries_select.value
    if 'All' in selected_countries:
        leagues_select.options = np.append('All', df_data['tour_name'].unique())
    else:
        leagues_select.options = np.append('All', df_data[df_data['country'].isin(selected_countries)]['tour_name'].unique())

# Register the update leagues function to be called when countries are selected
countries_select.observe(update_leagues, 'value')

# Create the plot function
def update_charts(country, leagues):
    if 'All' in country:
        filtered_data = df_data.copy()
    else:
        filtered_data = df_data[df_data['country'].isin(country)]

    if 'All' in leagues:
        filtered_data = filtered_data.copy()
    else:
        filtered_data = filtered_data[filtered_data['tour_name'].isin(leagues)]

    # Define the win_odds ranges
    win_odds_range = (1.8, 2.05)
    
    # Define the home_actual threshold values
    home_actual_thresholds = [(60, 70), (70, 75), (75, 80), (80, 100)]

    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the home_actual thresholds
    for i, home_actual_threshold in enumerate(home_actual_thresholds):
    
        # Determine the upper limit for the threshold range
        if i == len(home_actual_thresholds) - 1:
            upper_limit = filtered_data['home_actual'].max()
        else:
            upper_limit = home_actual_thresholds[i+1]
    
        # Filter the data based on the win_odds range and home_actual threshold
        display_data = filtered_data[(filtered_data['home_odds'] >= win_odds_range[0]) &
                                (filtered_data['home_odds'] < win_odds_range[1]) &
                                (filtered_data['home_actual'] >= home_actual_threshold[0]) &
                                (filtered_data['home_actual'] < home_actual_threshold[1]) &
                                (filtered_data['home_avg_rating'] > filtered_data['away_avg_rating'])]
    
        # Count the number of wins and losses/draws in the filtered data
        wins_count = display_data[display_data['home_team_win'] == 1]['home_team_win'].count()
        losses_draws_count = display_data[display_data['home_team_win'] == 0]['home_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Create a new figure
    plt.figure()

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(home_actual_thresholds)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(home_actual_thresholds)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')

    plt.xlabel('Home Actual Threshold')
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nWin Odds Range ({win_odds_range[0]} - {win_odds_range[1]})\nTotal Matches: {match_count}')
    plt.xticks(range(len(home_actual_thresholds)),[f'{home_actual_threshold[0]}% - {home_actual_threshold[1]}%' for home_actual_threshold in home_actual_thresholds])

    # Add a legend to the last subplot
    plt.legend()
    
    # Show the plot
    plt.show()

    # Close the figure to prevent multiple plots from being displayed
    plt.close()

# Define the apply button click event
def apply_button_clicked(b):
    selected_countries = countries_select.value
    selected_leagues = leagues_select.value
    with output:
        clear_output(wait=True)
        update_charts(selected_countries, selected_leagues)

# Display the widgets
display(widgets.HBox([countries_select, leagues_select]),apply_button, output)

# Register the apply button click event
apply_button.on_click(apply_button_clicked)

# Analysis of the Away Team Wins and Losses/Draws counts vs Away Actual Percentage data (> 60%, 70%, 75% and 80%) when Away Odds in the range 1.80 and 2.05 and Away Avg Rating > Home Avg Rating

In [ ]:
# Get the unique country and league names
unique_countries = df_data['country'].unique()

# Create the country and league selection widgets
countries_select = widgets.SelectMultiple(options=np.append('All', unique_countries), description='Countries:', rows=5)
leagues_select = widgets.SelectMultiple(options=[], description='Leagues:', rows=5)
apply_button = widgets.Button(description='Apply')
output = widgets.Output()

# Define the update leagues function
def update_leagues(*args):
    selected_countries = countries_select.value
    if 'All' in selected_countries:
        leagues_select.options = np.append('All', df_data['tour_name'].unique())
    else:
        leagues_select.options = np.append('All', df_data[df_data['country'].isin(selected_countries)]['tour_name'].unique())

# Register the update leagues function to be called when countries are selected
countries_select.observe(update_leagues, 'value')

# Create the plot function
def update_charts(country, leagues):
    if 'All' in country:
        filtered_data = df_data.copy()
    else:
        filtered_data = df_data[df_data['country'].isin(country)]

    if 'All' in leagues:
        filtered_data = filtered_data.copy()
    else:
        filtered_data = filtered_data[filtered_data['tour_name'].isin(leagues)]

    # Define the away_win_odds ranges
    away_odds_range = (1.8, 2.05)
    
    # Define the away_actual threshold values
    away_actual_thresholds = [(60, 70), (70, 75), (75, 80), (80, 100)]

    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the away_actual thresholds
    for i, away_actual_threshold in enumerate(away_actual_thresholds):

        # Filter the data based on the win_odds range and away_actual threshold
        display_data = filtered_data[(filtered_data['away_odds'] >= away_odds_range[0]) &
                                (filtered_data['away_odds'] < away_odds_range[1]) &
                                (filtered_data['away_actual'] >= away_actual_threshold[0]) &
                                (filtered_data['away_actual'] < away_actual_threshold[1]) &
                                (filtered_data['away_avg_rating'] > filtered_data['home_avg_rating'])]
    
        # Count the number of wins and losses/draws in the filtered data
        wins_count = display_data[display_data['away_team_win'] == 1]['away_team_win'].count()
        losses_draws_count = display_data[display_data['away_team_win'] == 0]['away_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Create a new figure
    plt.figure()

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(away_actual_thresholds)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(away_actual_thresholds)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')

    plt.xlabel('Away Actual Threshold')
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nWin Odds Range ({win_odds_range[0]} - {win_odds_range[1]})\nTotal Matches: {match_count}')
    plt.xticks(range(len(away_actual_thresholds)),[f'{away_actual_threshold[0]}% - {away_actual_threshold[1]}%' for away_actual_threshold in away_actual_thresholds])

    # Add a legend to the last subplot
    plt.legend()
    
    # Show the plot
    plt.show()

    # Close the figure to prevent multiple plots from being displayed
    plt.close()

# Define the apply button click event
def apply_button_clicked(b):
    selected_countries = countries_select.value
    selected_leagues = leagues_select.value
    with output:
        clear_output(wait=True)
        update_charts(selected_countries, selected_leagues)

# Display the widgets
display(widgets.HBox([countries_select, leagues_select]),apply_button, output)

# Register the apply button click event
apply_button.on_click(apply_button_clicked)

# Analysis of the Home Team Wins and Losses/Draws counts vs Home Actual Percentage data (< 50%, 40%, 30% and 20%) and Home Odds in the range 1.80 and 2.05

In [ ]:
# Get the unique country and league names
unique_countries = df_data['country'].unique()

# Create the country and league selection widgets
countries_select = widgets.SelectMultiple(options=np.append('All', unique_countries), description='Countries:', rows=5)
leagues_select = widgets.SelectMultiple(options=[], description='Leagues:', rows=5)
apply_button = widgets.Button(description='Apply')
output = widgets.Output()

# Define the update leagues function
def update_leagues(*args):
    selected_countries = countries_select.value
    if 'All' in selected_countries:
        leagues_select.options = np.append('All', df_data['tour_name'].unique())
    else:
        leagues_select.options = np.append('All', df_data[df_data['country'].isin(selected_countries)]['tour_name'].unique())

# Register the update leagues function to be called when countries are selected
countries_select.observe(update_leagues, 'value')

# Create the plot function
def update_charts(country, leagues):
    if 'All' in country:
        filtered_data = df_data.copy()
    else:
        filtered_data = df_data[df_data['country'].isin(country)]

    if 'All' in leagues:
        filtered_data = filtered_data.copy()
    else:
        filtered_data = filtered_data[filtered_data['tour_name'].isin(leagues)]

    # Define the win_odds ranges
    win_odds_range = (1.8, 2.05)
    
    # Define the home_actual threshold values
    home_actual_thresholds = [(0, 20), (20, 30), (30, 40), (40, 50)]

    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the home_actual thresholds
    for i, home_actual_threshold in enumerate(home_actual_thresholds):
    
        # Filter the data based on the win_odds range and home_actual threshold
        display_data = filtered_data[(filtered_data['home_odds'] >= win_odds_range[0]) &
                                (filtered_data['home_odds'] < win_odds_range[1]) &
                                (filtered_data['home_actual'] >= home_actual_threshold[0]) &
                                (filtered_data['home_actual'] < home_actual_threshold[1])]
    
        # Count the number of wins and losses/draws in the filtered data
        wins_count = display_data[display_data['home_team_win'] == 1]['home_team_win'].count()
        losses_draws_count = display_data[display_data['home_team_win'] == 0]['home_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Create a new figure
    plt.figure()

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(home_actual_thresholds)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(home_actual_thresholds)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')

    plt.xlabel('Home Actual Threshold')
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nWin Odds Range ({win_odds_range[0]} - {win_odds_range[1]})\nTotal Matches: {match_count}')
    plt.xticks(range(len(home_actual_thresholds)),[f'{home_actual_threshold[0]}% - {home_actual_threshold[1]}%' for home_actual_threshold in home_actual_thresholds])

    # Add a legend to the last subplot
    plt.legend()
    
    # Show the plot
    plt.show()

    # Close the figure to prevent multiple plots from being displayed
    plt.close()

# Define the apply button click event
def apply_button_clicked(b):
    selected_countries = countries_select.value
    selected_leagues = leagues_select.value
    with output:
        clear_output(wait=True)
        update_charts(selected_countries, selected_leagues)

# Display the widgets
display(widgets.HBox([countries_select, leagues_select]),apply_button, output)

# Register the apply button click event
apply_button.on_click(apply_button_clicked)

# Analysis of the Away Team Wins and Losses/Draws counts vs Away Actual Percentage data (< 50%, 40%, 30% and 20%) when Away Odds in the range 1.80 and 2.05

In [ ]:
# Get the unique country and league names
unique_countries = df_data['country'].unique()

# Create the country and league selection widgets
countries_select = widgets.SelectMultiple(options=np.append('All', unique_countries), description='Countries:', rows=5)
leagues_select = widgets.SelectMultiple(options=[], description='Leagues:', rows=5)
apply_button = widgets.Button(description='Apply')
output = widgets.Output()

# Define the update leagues function
def update_leagues(*args):
    selected_countries = countries_select.value
    if 'All' in selected_countries:
        leagues_select.options = np.append('All', df_data['tour_name'].unique())
    else:
        leagues_select.options = np.append('All', df_data[df_data['country'].isin(selected_countries)]['tour_name'].unique())

# Register the update leagues function to be called when countries are selected
countries_select.observe(update_leagues, 'value')

# Create the plot function
def update_charts(country, leagues):
    if 'All' in country:
        filtered_data = df_data.copy()
    else:
        filtered_data = df_data[df_data['country'].isin(country)]

    if 'All' in leagues:
        filtered_data = filtered_data.copy()
    else:
        filtered_data = filtered_data[filtered_data['tour_name'].isin(leagues)]

    # Define the away_win_odds ranges
    away_odds_range = (1.8, 2.05)
    
    # Define the away_actual threshold values
    away_actual_thresholds = [(0, 20), (20, 30), (30, 40), (40, 50)]

    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the away_actual thresholds
    for i, away_actual_threshold in enumerate(away_actual_thresholds):
    
        # Filter the data based on the win_odds range and away_actual threshold
        display_data = filtered_data[(filtered_data['away_odds'] >= away_odds_range[0]) &
                                (filtered_data['away_odds'] < away_odds_range[1]) &
                                (filtered_data['away_actual'] >= away_actual_threshold[0]) &
                                (filtered_data['away_actual'] < away_actual_threshold[1])]
    
        # Count the number of wins and losses/draws in the filtered data
        wins_count = display_data[display_data['away_team_win'] == 1]['away_team_win'].count()
        losses_draws_count = display_data[display_data['away_team_win'] == 0]['away_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Create a new figure
    plt.figure()

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(away_actual_thresholds)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(away_actual_thresholds)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')

    plt.xlabel('Away Actual Threshold')
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nWin Odds Range ({win_odds_range[0]} - {win_odds_range[1]})\nTotal Matches: {match_count}')
    plt.xticks(range(len(away_actual_thresholds)),[f'{away_actual_threshold[0]}% - {away_actual_threshold[1]}%' for away_actual_threshold in away_actual_thresholds])

    # Add a legend to the last subplot
    plt.legend()
    
    # Show the plot
    plt.show()

    # Close the figure to prevent multiple plots from being displayed
    plt.close()

# Define the apply button click event
def apply_button_clicked(b):
    selected_countries = countries_select.value
    selected_leagues = leagues_select.value
    with output:
        clear_output(wait=True)
        update_charts(selected_countries, selected_leagues)

# Display the widgets
display(widgets.HBox([countries_select, leagues_select]),apply_button, output)

# Register the apply button click event
apply_button.on_click(apply_button_clicked)

# Analysis of the Home Team Wins and Losses/Draws counts vs Home Actual Percentage data (< 50%, 40%, and 30% ) and Home Odds in the range 1.55 and 1.79 and Home Avg Rating <= Away Avg Rating

In [ ]:
# Get the unique country and league names
unique_countries = df_data['country'].unique()

# Create the country and league selection widgets
countries_select = widgets.SelectMultiple(options=np.append('All', unique_countries), description='Countries:', rows=5)
leagues_select = widgets.SelectMultiple(options=[], description='Leagues:', rows=5)
apply_button = widgets.Button(description='Apply')
output = widgets.Output()

# Define the update leagues function
def update_leagues(*args):
    selected_countries = countries_select.value
    if 'All' in selected_countries:
        leagues_select.options = np.append('All', df_data['tour_name'].unique())
    else:
        leagues_select.options = np.append('All', df_data[df_data['country'].isin(selected_countries)]['tour_name'].unique())

# Register the update leagues function to be called when countries are selected
countries_select.observe(update_leagues, 'value')

# Create the plot function
def update_charts(country, leagues):
    if 'All' in country:
        filtered_data = df_data.copy()
    else:
        filtered_data = df_data[df_data['country'].isin(country)]

    if 'All' in leagues:
        filtered_data = filtered_data.copy()
    else:
        filtered_data = filtered_data[filtered_data['tour_name'].isin(leagues)]

    # Define the win_odds ranges
    win_odds_range = (1.55, 1.79)
    
    # Define the home_actual threshold values
    home_actual_thresholds = [(0, 30), (30, 40), (40, 50)]

    wins_counts = []
    losses_draws_counts = []
    match_count = 0

    # Iterate over the home_actual thresholds
    for i, home_actual_threshold in enumerate(home_actual_thresholds):
    
        # Filter the data based on the win_odds range and home_actual threshold
        display_data = filtered_data[(filtered_data['home_odds'] >= win_odds_range[0]) &
                                (filtered_data['home_odds'] < win_odds_range[1]) &
                                (filtered_data['home_actual'] >= home_actual_threshold[0]) &
                                (filtered_data['home_actual'] < home_actual_threshold[1]) &
                                (filtered_data['home_avg_rating'] <= filtered_data['away_avg_rating'])]
    
        # Count the number of wins and losses/draws in the filtered data
        wins_count = display_data[display_data['home_team_win'] == 1]['home_team_win'].count()
        losses_draws_count = display_data[display_data['home_team_win'] == 0]['home_team_win'].count()
    
        # Append the counts to the respective lists
        wins_counts.append(wins_count)
        losses_draws_counts.append(losses_draws_count)
        match_count = match_count + wins_count + losses_draws_count

    # Create a new figure
    plt.figure()

    # Plot the stacked bar chart for the current threshold
    plt.bar(range(len(home_actual_thresholds)), wins_counts, label='Wins', alpha=0.7)
    plt.bar(range(len(home_actual_thresholds)), losses_draws_counts, bottom=wins_counts, label='Losses/Draws', alpha=0.7)

    # Display the counts within the stacked bar
    for j, (wins, losses_draws) in enumerate(zip(wins_counts, losses_draws_counts)):
        plt.text(j, wins / 2, str(wins), ha='center', va='center')
        plt.text(j, wins + losses_draws / 2, str(losses_draws), ha='center', va='center')

    plt.xlabel('Home Actual Threshold')
    plt.ylabel('Counts')
    plt.title(f'Counts of Wins and Losses/Draws\nWin Odds Range ({win_odds_range[0]} - {win_odds_range[1]})\nTotal Matches: {match_count}')
    plt.xticks(range(len(home_actual_thresholds)),[f'{home_actual_threshold[0]}% - {home_actual_threshold[1]}%' for home_actual_threshold in home_actual_thresholds])

    # Add a legend to the last subplot
    plt.legend()
    
    # Show the plot
    plt.show()

    # Close the figure to prevent multiple plots from being displayed
    plt.close()

# Define the apply button click event
def apply_button_clicked(b):
    selected_countries = countries_select.value
    selected_leagues = leagues_select.value
    with output:
        clear_output(wait=True)
        update_charts(selected_countries, selected_leagues)

# Display the widgets
display(widgets.HBox([countries_select, leagues_select]),apply_button, output)

# Register the apply button click event
apply_button.on_click(apply_button_clicked)